In [117]:
!/home/jovyan/work/usr/bin/kubectl get pods
#!POD_NAME=$(/home/jovyan/work/usr/bin/kubectl get pod -l app=peer0-org1-scray-org | tail -n 1 | cut -d' ' -f1)
#!echo $POD_NAME

NAME                                               READY   STATUS    RESTARTS   AGE
hl-fabric-data-share-deployment-56d49c4d9b-d9trf   1/1     Running   0          56d
invoice-chaincode-external-6566b486ff-zcl8c        1/1     Running   0          18h
peer0-org1-scray-org-5c4d69f556-snz5c              2/2     Running   0          3h6m


# Delete old peer

In [109]:
!/home/jovyan/work/usr/bin/kubectl delete configmap hl-fabric-peer 
!/home/jovyan/work/usr/bin/kubectl delete -f https://raw.githubusercontent.com/scray/scray/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/k8s-peer.yaml

configmap "hl-fabric-peer" deleted
deployment.apps "peer0-org1-scray-org" deleted
service "peer0-org1-scray-org" deleted


# Create new peer and 'apply'

In [112]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=OrgSally \ 
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:30003 \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:30001 \
 --from-literal=CORE_PEER_LOCALMSPID=OrgSallyMSP

!/home/jovyan/work/usr/bin/kubectl apply -f https://raw.githubusercontent.com/scray/scray/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/k8s-peer.yaml        

configmap/hl-fabric-peer created
deployment.apps/peer0-org1-scray-org created
service/peer0-org1-scray-org created


1. Der neue Peer wurde erzeugt. All relevant infos and cryptos for the application is uploaded to external sharepoint
2. Now the application needs to be signed by other peers

# Now the peer can join the network

In [114]:
import os
import json
import subprocess
import pandas as pd

!/home/jovyan/work/usr/bin/kubectl get pods
os.environ['POD_NAME'] = 'peer0-org1-scray-org-5c4d69f556-snz5c'

NAME                                               READY   STATUS    RESTARTS   AGE
hl-fabric-data-share-deployment-56d49c4d9b-d9trf   1/1     Running   0          56d
invoice-chaincode-external-6566b486ff-zcl8c        1/1     Running   0          14h
peer0-org1-scray-org-5c4d69f556-snz5c              2/2     Running   0          2m24s


In [115]:
#!/home/jovyan/work/usr/bin/kubectl get pods | fgrep peer0
#!POD_NAME=$(/home/jovyan/work/usr/bin/kubectl get pod -l app=peer0-org1-scray-org | tail -n 1 | cut -d' ' -f1)
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!echo $POD_NAME
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP $ORDERER_HOSTNAME $CHANNEL_NAME

peer0-org1-scray-org-5c4d69f556-snz5c
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz
(1/1) Installing curl (7.69.1-r1)
7  0%                                                                           87100% ##########################################################################8Executing busybox-1.31.1-r16.trigger
OK: 29 MiB in 28 packages
(1/17) Installing fstrm (0.6.0-r1)
7  0%                                                                           8(2/17) Installing libgcc (9.3.0-r2)
7  0%                                                                           8(3/17) Installing krb5-conf (1.0-r2)
7  1%                                                                           8(4/17) Installing libcom_err (1.45.6-r0)
7  1%                                                                           8(5/17) Installing keyutils-libs (1.6.1-r1)
7  1% #        

In [68]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty peer0-org1-scray-org-84ddc5757f-glbgn -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Install and aprove chaincode

In [116]:
# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   1250      0 --:--:-- --:--:-- --:--:--  1246
2020-12-08 09:59:13.197 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-08 09:59:13.198 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2020-12-08 09:59:15.388 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [a084f06991b010f25c19cf88afdabc36811439958675b0fafb824216c9648114] committed with status (VALID) at peer0.kubernetes.research.dev.seeburger.de:30003
